# Everybody Dance Now
Let's make a dancing figure using control net


## Install denpendencies

In [4]:
!pip install -q controlnet_aux transformers accelerate
!pip install -q git+https://github.com/huggingface/diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Running Control Net

### First get the pose estimator from control net

In [ ]:
from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image

openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")

Here's an example of what it does:

In [ ]:
openpose_image = load_image(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/person.png"
)
openpose_image.show()

In [ ]:
openpose_image = openpose(openpose_image)

import matplotlib.pyplot as plt

openpose_image.show()

### Next, get the control net image generator
You probably need an A-100 GPU for this. In case of errors while using other types of GPUs, go to stackoverflow.com, ask google, or checkout diffuser's github repository.

In [ ]:
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel, AutoencoderKL, UniPCMultistepScheduler
import torch

# Initialize ControlNet pipeline.
controlnet = ControlNetModel.from_pretrained("thibaud/controlnet-openpose-sdxl-1.0", torch_dtype=torch.float16)
pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", controlnet=controlnet, torch_dtype=torch.float16
)
pipe.enable_model_cpu_offload()

### Get your data

Connect to your google drive. You should have uploaded a video to your drive at this point. If you haven't, upload a video now.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Extract the video into image frames using ffmpeg

In [2]:
!mkdir -p frames
!rm frames/*
# Usage: !ffmpeg -i <path_to_your_uploaded_video> -vf "fps=25" frames/frame%06d.png
!ffmpeg -i /content/drive/MyDrive/cai.mp4 -vf "fps=25" frames/frame%06d.png

rm: cannot remove 'frames/*': No such file or directory
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --ena

In [3]:
!ls /content/frames | wc -l

1479


### Start generating!

In [ ]:
!mkdir -p /content/drive/MyDrive/results/

In [ ]:
start_frame = 600
end_frame = 900
prompt = "Spider man dancing in a desert, high quality, cartoon style"  # what you want to generate
negative_prompt = "low quality, bad quality"  # what you don't want to generate

for idx, i in enumerate(range(start_frame, end_frame)):
    # openpose_image = load_image(f"/content/vid1/frame{i:06d}.png")
    openpose_image = load_image(f"/content/frames/frame{i:06d}.png")
    openpose_image = openpose(openpose_image)

    images = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=25,
        num_images_per_prompt=1,
        image=openpose_image,
        generator=torch.manual_seed(97),
    ).images
    images[0].save(f"/content/drive/MyDrive/results/frame{idx+1:06d}.png")
    # if i == 10:
    #     break

Assemble the final video from the generated frames

In [ ]:
# Usage: !ffmpeg -framerate 25 -i <path_to_generate_frames> -c:v libx264 -movflags +faststart -pix_fmt yuv420p <path_to_result_video>
!ffmpeg -framerate 25 -i /content/drive/MyDrive/results/frame%06d.png -c:v libx264 -movflags +faststart -pix_fmt yuv420p /content/drive/MyDrive/results/results.mp4